# Neural Style Transfer Experiment
This notebook demonstrates style transfer using the backbone implementation.

In [ ]:
import sys
sys.path.append('../src')
from utils import load_image, imshow
content_img = load_image('../data/content.jpg')
style_img = load_image('../data/style.jpg')
imshow(content_img, title='Content Image')
imshow(style_img, title='Style Image')

In [ ]:
# Run style transfer
import subprocess
subprocess.run(['python', '../src/style_transfer.py', '--content', '../data/content.jpg', '--style', '../data/style.jpg', '--output', '../outputs/result.png'])

In [ ]:
# Show result
from PIL import Image
import matplotlib.pyplot as plt
result = Image.open('../outputs/result.png')
plt.imshow(result)
plt.title('Result Image')
plt.show()